# Capacity prediction
### Notebook 2: Entries and occupancies data
From the dataset compiled in the previous notebook, adds entries and occupancies data while also calculating fundamental statistical measures.

author: acg

In [1]:
import pandas as pd
import sys
sys.path.append("..")
from utils.configuration import (HOST_PERSEO_PRE as HOST, DWH_DATABASE, UID, PWD_PRE as PWD)
from utils.configuration import (HOST_PERSEO_PRO as HOST_PRO, DWH_DATABASE, UID, PWD_PRO as PWD_PRO)
from utils.dwh_connection import (get_dwh_connection)
import warnings
warnings.filterwarnings('ignore')

In [2]:
from database.repository import select_ent_occ_groupby_sector_day,select_meteo_events_groupby_day
from utils.properties import * 

In [3]:
# import dataset created in notebook1
filepath = OUTPUT_PATH + "\\" + OUTPUT_FILENAME_SECTORS
df_all_sectors = pd.read_csv(filepath, sep=";")

connection_dwh = get_dwh_connection(HOST_PRO, DWH_DATABASE, UID, PWD_PRO)
date_from = '2022-01-01'
date_to = '2023-01-01'

In [4]:
pd.set_option('display.max_columns', None)
df_all_sectors.head(8)

,LevelType,ATCUnitCode,SectorCode,ATCType,Polygon_area,Perimeter,NumberOfVertices,Centroid_Latitude,Centroid_Longitude,Capacity,Date_From,Date_To,approximation_route,lowest_bound,highest_bound,ground,ceiling,from_ground_to_ceiling,volume_with_steps,volume_lowest_highest,volume_airblocks_avg,num_airblocks_avg,Num_fluxes,all_attitudesIN,all_attitudesOUT,attitude_cruise_cruise,attitude_cruise_descend_comb,attitude_cruise_climb_comb,attitude_decent_descend,attitude_climb_climb,attitude_descend_climb_comb,sum_nFlights,max_nFlights,avg_nFlights,median_nFlights,std_nFlights,percentile25_nFlights,percentile75_nFlights,sum_avgTimeInRoute,max_avgTimeInRoute,median_avgTimeInRoute,std_avgTimeInRoute,sum_avgTimeInRoute_divided_num_flight,percentile25_avgTimeInRoute,percentile75_avgTimeInRoute,all_latIN_fluxes,all_lngIN_fluxes,all_latOUT_fluxes,all_lngOUT_fluxes,avg_latIN_fluxes,avg_lngIN_fluxes,avg_latOUT_fluxes,avg_lngOUT_fluxes,num_interacting_fluxes,num_interacting_fluxes_type_0,num_interacting_fluxes_type_1,num_interacting_fluxes_type_2,num_interacting_fluxes_type_3,avg_interacting_type,median_interacting_type,max_num_flights_interacting_fluxes_type_1,avg_num_flights_interacting_fluxes_type_1,median_num_flights_interacting_fluxes_type_1,std_num_flights_interacting_fluxes_type_1,percentile25_num_flights_interacting_fluxes_type_1,percentile75_num_flights_interacting_fluxes_type_1,max_num_flights_interacting_fluxes_type_2,avg_num_flights_interacting_fluxes_type_2,median_num_flights_interacting_fluxes_type_2,std_num_flights_interacting_fluxes_type_2,percentile25_num_flights_interacting_fluxes_type_2,percentile75_num_flights_interacting_fluxes_type_2,max_num_flights_interacting_fluxes_type_3,avg_num_flights_interacting_fluxes_type_3,median_num_flights_interacting_fluxes_type_3,std_num_flights_interacting_fluxes_type_3,percentile25_num_flights_interacting_fluxes_type_3,percentile75_num_flights_interacting_fluxes_type_3,max_avg_time_route_type_1_combined,sum_avg_time_route_type_1_divided_num_flight_combined,median_avg_time_route_type_1_combined,std_avg_time_route_type_1_combined,percentile25_avg_time_route_type_1_combined,percentile75_avg_time_route_type_1_combined,max_avg_time_route_type_2_combined,sum_avg_time_route_type_2_divided_num_flight_combined,median_avg_time_route_type_2_combined,std_avg_time_route_type_2_combined,percentile25_avg_time_route_type_2_combined,percentile75_avg_time_route_type_2_combined,max_avg_time_route_type_3_combined,sum_avg_time_route_type_3_divided_num_flight_combined,median_avg_time_route_type_3_combined,std_avg_time_route_type_3_combined,percentile25_avg_time_route_type_3_combined,percentile75_avg_time_route_type_3_combined,max_complexity_type_None,sum_complexity_type_None,median_complexity_type_None,std_complexity_type_None,percentile25_complexity_type_None,percentile75_complexity_type_None,max_complexity_type_1,sum_complexity_type_1,median_complexity_type_1,std_complexity_type_1,percentile25_complexity_type_1,percentile75_complexity_type_1,max_complexity_type_2,sum_complexity_type_2,median_complexity_type_2,std_complexity_type_2,percentile25_complexity_type_2,percentile75_complexity_type_2,max_complexity_type_3,sum_complexity_type_3,median_complexity_type_3,std_complexity_type_3,percentile25_complexity_type_3,percentile75_complexity_type_3,max_harmonic_complexity_type_None,sum_harmonic_complexity_type_None,median_harmonic_complexity_type_None,std_harmonic_complexity_type_None,percentile25_harmonic_complexity_type_None,percentile75_harmonic_complexity_type_None,max_harmonic_complexity_type_1,sum_harmonic_complexity_type_1,median_harmonic_complexity_type_1,std_harmonic_complexity_type_1,percentile25_harmonic_complexity_type_1,percentile75_harmonic_complexity_type_1,max_harmonic_complexity_type_2,sum_harmonic_complexity_type_2,median_harmonic_complexity_type_2,std_harmonic_complexity_type_2,percentile25_harmonic_complexity_type_2,percentile75_harmonic_complexity_type_2,max_harmonic_complexity_type_3,sum_harmonic

In [5]:
df_all_sectors.columns

Index(['LevelType', 'ATCUnitCode', 'SectorCode', 'ATCType', 'Polygon_area',
       'Perimeter', 'NumberOfVertices', 'Centroid_Latitude',
       'Centroid_Longitude', 'Capacity',
       ...
       'median_complexity', 'std_complexity', 'percentile25_complexity',
       'percentile75_complexity', 'max_harmonic_complexity',
       'sum_harmonic_complexity', 'median_harmonic_complexity',
       'std_harmonic_complexity', 'percentile25_harmonic_complexity',
       'percentile75_harmonic_complexity'],
      dtype='object', length=156)

In [6]:
df_all_sectors = df_all_sectors.rename(columns={
    'LevelType': 'level_type',
    'ATCUnitCode':'atcunit_code',
    'ATCType':'atc_type',
    'SectorCode': 'sector_code',
    'Date_From': 'date_from',
    'Date_To': 'date_to',
    'Polygon_area': 'sector_area',
    'Perimeter': 'perimeter',
    'NumberOfVertices': 'num_vertices',
    'Centroid_Latitude': 'centroid_lat',
    'Centroid_Longitude': 'centroid_lon',
    'Capacity': 'capacity',
    'Num_fluxes': 'num_fluxes'
})

### Filter (not implemented)

# Entries and occupancies queries

Average entries and occupancies by sector and day

In [7]:
df_ent_occ = select_ent_occ_groupby_sector_day(connection_dwh, date_from, date_to)
df_ent_occ.head(8)

,dateKey,trafficVolumeCode,Date_From,Date_To,max_occupancyInWindow_1,occupancyInWindow_1,entriesInWindow_1,max_entriesInWindow_1,occupancyInWindow_5,max_occupancyInWindow_5,entriesInWindow_5,max_entriesInWindow_5,occupancyInWindow_10,max_occupancyInWindow_10,entriesInWindow_10,max_entriesInWindow_10,entriesInWindow_60,max_entriesInWindow_60
0,20220101,GCCCAAC,2021-08-12,2022-01-21,10,4,0,2,5,11,1,5,7,15,3,10,21,35
1,20220102,GCCCAAC,2021-08-12,2022-01-21,11,3,0,2,5,13,1,4,6,15,3,8,19,37
2,20220103,GCCCAAC,2021-08-12,2022-01-21,10,3,0,2,4,13,1,6,6,14,2,10,18,39
3,20220104,GCCCAAC,2021-08-12,2022-01-21,10,3,0,2,4,13,1,4,6,15,3,7,18,37
4,20220105,GCCCAAC,2021-08-12,2022-01-21,7,2,0,2,3,9,1,4,4,11,2,6,13,29
5,20220106,GCCCAAC,2021-08-12,2022-01-21,9,2,0,2,3,9,1,4,5,11,2,7,15,29
6,20220107,GCCCAAC,2021-08-12,2022-01-21,7,3,0,2,3,10,1,6,5,14,2,10,14,30
7,20220108,GCCCAAC,2021-08-12,2022-01-21,10,4,0,2,6,12,1,6,7,15,3,9,23,40


# Meteo data

Esta información es complicada de integrar al momento, porque:
1. Hay pocos datos
2. No hay información meteo a nivel de ruta al momento

In [8]:
df_meteo = select_meteo_events_groupby_day(connection_dwh, 'LEBL', date_from, date_to)
df_meteo.head(8)

,datekey,avg_temperature,avg_atmpressure,avg_wind_speed,avg_cloud_height,event,intensity
0,2022-01-01,10,1026,5,7.0,Neblina,Moderado
1,2022-01-02,10,1027,4,7.0,Niebla,Moderado
2,2022-01-03,11,1023,8,7.0,Nieve,Baja
3,2022-01-04,12,1009,5,11.0,Nieve,Moderado
4,2022-01-05,10,1006,9,22.0,Lluvia,Baja
5,2022-01-06,6,1016,10,34.0,Nieve,Baja
6,2022-01-07,6,1022,6,24.0,Nieve,Baja
7,2022-01-08,8,1020,8,27.0,Lluvia,Baja


### Add statistic features

For each period we aggregate to compute the average an percentiles, while also renaming the columns

In [9]:
# complex aggregation
columns_to_agg = [
    'occupancyInWindow_1', 'entriesInWindow_1',
    'occupancyInWindow_5', 'entriesInWindow_5',
    'occupancyInWindow_10', 'entriesInWindow_10',
    'entriesInWindow_60'
]
max_columns = [
    'max_occupancyInWindow_1', 'max_entriesInWindow_1',
    'max_occupancyInWindow_5', 'max_entriesInWindow_5',
    'max_occupancyInWindow_10', 'max_entriesInWindow_10',
    'max_entriesInWindow_60'
]

agg_funcs = {}
for col in columns_to_agg:
    agg_funcs[col] = ['mean', lambda x: x.quantile(0.25), 'median', lambda x: x.quantile(0.75)]

for col in max_columns:
    agg_funcs[col] = 'max'

# group by workShiftCodes 
df_ent_occ_grouped = df_ent_occ.groupby(['trafficVolumeCode', 'Date_From', 'Date_To']).agg(agg_funcs)

In [10]:
# df_ent_occ_grouped.columns = [
#     f'avg_{col}' if func == 'mean' else 
#     f'percentile25_{col}' if func == '<lambda_0>' else 
#     f'median_{col}' if func == 'median' else 
#     f'percentile75_{col}' for col, func in df_ent_occ_grouped.columns
# ]
# df_ent_occ_grouped = df_ent_occ_grouped.reset_index()

df_ent_occ_grouped.columns = [
    f'avg_{col}' if func == 'mean' else 
    f'percentile25_{col}' if func == '<lambda_0>' else 
    f'percentile75_{col}' if func == '<lambda_1>' else 
    f'median_{col}' if func == 'median' else 
    f'{col}' if func == 'max' else col
    for col, func in df_ent_occ_grouped.columns
]
df_ent_occ_grouped = df_ent_occ_grouped.reset_index()



In [11]:
df_ent_occ_grouped

,trafficVolumeCode,Date_From,Date_To,avg_occupancyInWindow_1,percentile25_occupancyInWindow_1,median_occupancyInWindow_1,percentile75_occupancyInWindow_1,avg_entriesInWindow_1,percentile25_entriesInWindow_1,median_entriesInWindow_1,percentile75_entriesInWindow_1,avg_occupancyInWindow_5,percentile25_occupancyInWindow_5,median_occupancyInWindow_5,percentile75_occupancyInWindow_5,avg_entriesInWindow_5,percentile25_entriesInWindow_5,median_entriesInWindow_5,percentile75_entriesInWindow_5,avg_occupancyInWindow_10,percentile25_occupancyInWindow_10,median_occupancyInWindow_10,percentile75_occupancyInWindow_10,avg_entriesInWindow_10,percentile25_entriesInWindow_10,median_entriesInWindow_10,percentile75_entriesInWindow_10,avg_entriesInWindow_60,percentile25_entriesInWindow_60,median_entriesInWindow_60,percentile75_entriesInWindow_60,max_occupancyInWindow_1,max_entriesInWindow_1,max_occupancyInWindow_5,max_entriesInWindow_5,max_occupancyInWindow_10,max_entriesInWindow_10,max_entriesInWindow_60
0,GCCCAAC,2021-08-12,2022-01-21,2.619048,2.0,3.0,3.0,0.000000,0.0,0.0,0.0,3.571429,3.0,3.0,4.0,0.666667,0.0,1.0,1.0,4.904762,4.0,5.0,6.0,1.952381,2.0,2.0,2.0,14.666667,12.0,13.0,18.0,11,2,13,6,15,10,40
1,GCCCAAC,2022-01-21,2022-12-13,3.370031,3.0,3.0,4.0,0.000000,0.0,0.0,0.0,4.538226,4.0,4.0,5.0,0.972477,1.0,1.0,1.0,6.012232,5.0,6.0,7.0,2.382263,2.0,2.0,3.0,17.000000,15.0,17.0,19.0,14,5,17,8,22,12,44
2,GCCCAAC,2022-12-13,9999-12-31,3.684211,3.0,4.0,4.0,0.000000,0.0,0.0,0.0,5.052632,5.0,5.0,5.5,1.105263,1.0,1.0,1.0,6.526316,6.0,7.0,7.0,2.789474,2.0,3.0,3.0,19.105263,18.0,19.0,20.5,13,3,16,6,19,10,42
3,GCCCACW,2021-08-12,2022-01-21,1.142857,1.0,1.0,1.0,0.000000,0.0,0.0,0.0,1.761905,2.0,2.0,2.0,0.047619,0.0,0.0,0.0,2.380952,2.0,2.0,3.0,0.952381,1.0,1.0,1.0,7.714286,7.0,8.0,8.0,7,3,9,5,10,8,24
4,GCCCACW,2022-01-21,2022-07-14,1.360000,1.0,1.0,2.0,0.000000,0.0,0.0,0.0,2.074286,2.0,2.0,2.0,0.051429,0.0,0.0,0.0,2.760000,2.0,3.0,3.0,1.028571,1.0,1.0,1.0,8.154286,7.0,8.0,9.0,9,3,10,6,12,8,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,LEMGANM,2018-05-31,2023-04-20,1.627397,1.0,2.0,2.0,0.000000,0.0,0.0,0.0,2.715068,2.0,3.0,4.0,0.726027,0.0,1.0,1.0,4.027397,3.0,4.0,5.0,2.156164,1.0,2.0,3.0,15.378082,11.0,16.0,20.0,11,3,15,8,19,12,58
763,LEMGAPM,2018-05-31,2023-04-20,2.980707,2.0,3.0,4.0,0.003215,0.0,0.0,0.0,4.202572,3.0,5.0,5.0,0.983923,1.0,1.0,1.0,5.778135,5.0,6.0,7.0,2.591640,2.0,3.0,3.0,18.157556,15.0,20.0,21.0,15,4,19,10,22,18,80
764,LEMGASM,2018-05-31,2023-04-20,1.317507,1.0,1.0,2.0,0.002967,0.0,0.0,0.0,2.065282,1.0,2.0,3.0,0.448071,0.0,0.0,1.0,2.997033,2.0,3.0,4.0,1.415430,1.0,1.0,2.0,10.667656,9.0,11.0,13.0,10,3,12,8,15,12,48
765,LEMGDNM,2018-05-31,2023-04-20,1.659164,1.0,2.0,2.0,0.000000,0.0,0.0,0.0,2.887460,2.0,3.0,3.0,0.954984,1.0,1.0,1.0,4.389068,4.0,5.0,5.0,2.546624,2.0,3.0,3.0,17.781350,15.0,19.0,21.0,12,4,15,8,19,16,78


In [12]:
df_ent_occ_grouped = df_ent_occ_grouped.rename(columns={
    'trafficVolumeCode': 'sector_code',
    'Date_From': 'date_from',
    'Date_To': 'date_to',
    'avg_occupancyInWindow_1': 'avg_occ_w1',
    'percentile25_occupancyInWindow_1': 'Q25_occ_w1',
    'median_occupancyInWindow_1': 'median_occ_w1',
    'percentile75_occupancyInWindow_1': 'Q75_occ_w1',
    'avg_entriesInWindow_1': 'avg_ent_w1',
    'percentile25_entriesInWindow_1': 'Q25_ent_w1',
    'median_entriesInWindow_1': 'median_ent_w1',
    'percentile75_entriesInWindow_1': 'Q75_ent_w1',
    'avg_occupancyInWindow_5': 'avg_occ_w5',
    'percentile25_occupancyInWindow_5': 'Q25_occ_w5',
    'median_occupancyInWindow_5': 'median_occ_w5',
    'percentile75_occupancyInWindow_5': 'Q75_occ_w5',
    'avg_entriesInWindow_5': 'avg_ent_w5',
    'percentile25_entriesInWindow_5': 'Q25_ent_w5',
    'median_entriesInWindow_5': 'median_ent_w5',
    'percentile75_entriesInWindow_5': 'Q75_ent_5',
    'avg_occupancyInWindow_10': 'avg_occ_w10',
    'percentile25_occupancyInWindow_10': 'Q25_occ_w10',
    'median_occupancyInWindow_10': 'median_occ_w10',
    'percentile75_occupancyInWindow_10': 'Q75_occ_w10',
    'avg_entriesInWindow_10': 'avg_ent_w10',
    'percentile25_entriesInWindow_10': 'Q25_ent_w10',
    'median_entriesInWindow_10': 'median_ent_w10',
    'percentile75_entriesInWindow_10': 'Q75_ent_w10',
    'avg_entriesInWindow_60': 'avg_ent_w60',
    'percentile25_entriesInWindow_60': 'Q25_ent_w60',
    'median_entriesInWindow_60': 'median_ent_w60',
    'percentile75_entriesInWindow_60': 'Q75_ent_w60'
})

# Merge dataframes

In [13]:
print(df_all_sectors[['sector_code', 'date_from', 'date_to']].drop_duplicates().reset_index(drop=True))
print(df_ent_occ_grouped[['sector_code', 'date_from', 'date_to']].drop_duplicates().reset_index(drop=True))

    sector_code   date_from     date_to
0       GCCCAAC  2021-08-12  2022-01-21
1       GCCCAAC  2022-01-21  2022-12-13
2       GCCCAAC  2022-12-13  2262-04-11
3       GCCCACW  2021-08-12  2022-01-21
4       GCCCACW  2022-01-21  2022-07-14
..          ...         ...         ...
789     LEMGANM  2018-05-31  2023-04-20
790     LEMGAPM  2018-05-31  2023-04-20
791     LEMGASM  2018-05-31  2023-04-20
792     LEMGDNM  2018-05-31  2023-04-20
793     LEMGDSM  2018-05-31  2023-04-20

[794 rows x 3 columns]
    sector_code   date_from     date_to
0       GCCCAAC  2021-08-12  2022-01-21
1       GCCCAAC  2022-01-21  2022-12-13
2       GCCCAAC  2022-12-13  9999-12-31
3       GCCCACW  2021-08-12  2022-01-21
4       GCCCACW  2022-01-21  2022-07-14
..          ...         ...         ...
762     LEMGANM  2018-05-31  2023-04-20
763     LEMGAPM  2018-05-31  2023-04-20
764     LEMGASM  2018-05-31  2023-04-20
765     LEMGDNM  2018-05-31  2023-04-20
766     LEMGDSM  2018-05-31  2023-04-20

[767 rows x 3 c

In [14]:
# Normalize all data types before merging
df_all_sectors['sector_code'] = df_all_sectors['sector_code'].str.strip()
df_ent_occ_grouped['sector_code'] = df_ent_occ_grouped['sector_code'].str.strip()

df_all_sectors['date_from'] = df_all_sectors['date_from'].str.strip()
df_ent_occ_grouped['date_from'] = df_ent_occ_grouped['date_from'].astype(str).str.strip()

df_all_sectors['date_to'] = df_all_sectors['date_to'].str.strip()
df_ent_occ_grouped['date_to'] = df_ent_occ_grouped['date_to'].astype(str).str.strip()

In [15]:
df = pd.merge(df_all_sectors, 
                      df_ent_occ_grouped, 
                      on=['sector_code', 'date_from', 'date_to'], how='inner')

In [16]:
df.head(8)

,level_type,atcunit_code,sector_code,atc_type,sector_area,perimeter,num_vertices,centroid_lat,centroid_lon,capacity,date_from,date_to,approximation_route,lowest_bound,highest_bound,ground,ceiling,from_ground_to_ceiling,volume_with_steps,volume_lowest_highest,volume_airblocks_avg,num_airblocks_avg,num_fluxes,all_attitudesIN,all_attitudesOUT,attitude_cruise_cruise,attitude_cruise_descend_comb,attitude_cruise_climb_comb,attitude_decent_descend,attitude_climb_climb,attitude_descend_climb_comb,sum_nFlights,max_nFlights,avg_nFlights,median_nFlights,std_nFlights,percentile25_nFlights,percentile75_nFlights,sum_avgTimeInRoute,max_avgTimeInRoute,median_avgTimeInRoute,std_avgTimeInRoute,sum_avgTimeInRoute_divided_num_flight,percentile25_avgTimeInRoute,percentile75_avgTimeInRoute,all_latIN_fluxes,all_lngIN_fluxes,all_latOUT_fluxes,all_lngOUT_fluxes,avg_latIN_fluxes,avg_lngIN_fluxes,avg_latOUT_fluxes,avg_lngOUT_fluxes,num_interacting_fluxes,num_interacting_fluxes_type_0,num_interacting_fluxes_type_1,num_interacting_fluxes_type_2,num_interacting_fluxes_type_3,avg_interacting_type,median_interacting_type,max_num_flights_interacting_fluxes_type_1,avg_num_flights_interacting_fluxes_type_1,median_num_flights_interacting_fluxes_type_1,std_num_flights_interacting_fluxes_type_1,percentile25_num_flights_interacting_fluxes_type_1,percentile75_num_flights_interacting_fluxes_type_1,max_num_flights_interacting_fluxes_type_2,avg_num_flights_interacting_fluxes_type_2,median_num_flights_interacting_fluxes_type_2,std_num_flights_interacting_fluxes_type_2,percentile25_num_flights_interacting_fluxes_type_2,percentile75_num_flights_interacting_fluxes_type_2,max_num_flights_interacting_fluxes_type_3,avg_num_flights_interacting_fluxes_type_3,median_num_flights_interacting_fluxes_type_3,std_num_flights_interacting_fluxes_type_3,percentile25_num_flights_interacting_fluxes_type_3,percentile75_num_flights_interacting_fluxes_type_3,max_avg_time_route_type_1_combined,sum_avg_time_route_type_1_divided_num_flight_combined,median_avg_time_route_type_1_combined,std_avg_time_route_type_1_combined,percentile25_avg_time_route_type_1_combined,percentile75_avg_time_route_type_1_combined,max_avg_time_route_type_2_combined,sum_avg_time_route_type_2_divided_num_flight_combined,median_avg_time_route_type_2_combined,std_avg_time_route_type_2_combined,percentile25_avg_time_route_type_2_combined,percentile75_avg_time_route_type_2_combined,max_avg_time_route_type_3_combined,sum_avg_time_route_type_3_divided_num_flight_combined,median_avg_time_route_type_3_combined,std_avg_time_route_type_3_combined,percentile25_avg_time_route_type_3_combined,percentile75_avg_time_route_type_3_combined,max_complexity_type_None,sum_complexity_type_None,median_complexity_type_None,std_complexity_type_None,percentile25_complexity_type_None,percentile75_complexity_type_None,max_complexity_type_1,sum_complexity_type_1,median_complexity_type_1,std_complexity_type_1,percentile25_complexity_type_1,percentile75_complexity_type_1,max_complexity_type_2,sum_complexity_type_2,median_complexity_type_2,std_complexity_type_2,percentile25_complexity_type_2,percentile75_complexity_type_2,max_complexity_type_3,sum_complexity_type_3,median_complexity_type_3,std_complexity_type_3,percentile25_complexity_type_3,percentile75_complexity_type_3,max_harmonic_complexity_type_None,sum_harmonic_complexity_type_None,median_harmonic_complexity_type_None,std_harmonic_complexity_type_None,percentile25_harmonic_complexity_type_None,percentile75_harmonic_complexity_type_None,max_harmonic_complexity_type_1,sum_harmonic_complexity_type_1,median_harmonic_complexity_type_1,std_harmonic_complexity_type_1,percentile25_harmonic_complexity_type_1,percentile75_harmonic_complexity_type_1,max_harmonic_complexity_type_2,sum_harmonic_complexity_type_2,median_harmonic_complexity_type_2,std_harmonic_complexity_type_2,percentile25_harmonic_complexity_type_2,percentile75_harmonic_complexity_type_2,max_harmonic_complexity_type_3,sum_harmonic_complexity_

In [17]:
# entries Q25 in w1 will be composed of zeroes only
df['Q25_occ_w1']

0      2.0
1      3.0
2      1.0
3      1.0
4      1.0
      ... 
583    1.0
584    2.0
585    1.0
586    1.0
587    1.0
Name: Q25_occ_w1, Length: 588, dtype: float64

In [19]:
df['Q25_ent_w1']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
583    0.0
584    0.0
585    0.0
586    0.0
587    0.0
Name: Q25_ent_w1, Length: 588, dtype: float64

# Save dataset

In [18]:
df.to_csv(FINAL_DATASET, sep=";", index=False)